In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# import libraries
import numpy as np
import pandas as pd 
import tensorflow as tf
from dateutil.parser import parse
dateparse=lambda dates:parse(dates)
import matplotlib.pyplot as plt

In [3]:
# load data from csv file and parse dates 
train_data = pd.read_csv('../input/daily-climate-time-series-data/DailyDelhiClimateTrain.csv',
                         parse_dates=['date'],date_parser=dateparse)
train_data

In [4]:
test_data = pd.read_csv('../input/daily-climate-time-series-data/DailyDelhiClimateTest.csv',
                         parse_dates=['date'],date_parser=dateparse)
test_data

In [5]:
# check cloumn data types
print(train_data.dtypes)
print(test_data.dtypes)

In [6]:
# convert data and menatemp column into numpy arraies for both test and training dfs
train_series = np.array(train_data.meantemp)
train_time = np.array(train_data.date)
test_series = np.array(test_data.meantemp)
test_time = np.array(test_data.date)

In [7]:
# print shapes 
print(train_series.shape)
print(test_series.shape)
print(train_time.shape)
print(test_time.shape)

In [8]:
# plot meantep and date columns 
plt.figure(figsize=(10, 6))
plt.plot(train_time,train_series)
plt.show()

In [9]:
window_size = 33 # 33 time steps that will be used as features to predict the next time step in the time series 
batch_size = 50 # the number of training examples in one forward/backward pass throught the network.
                # 1000/50 = 20 groups of 50 will be proagated throught the network 
shuffle_buffer_size = 500 # shuffle data so that its not in any order 

In [10]:
# convert dataset into training data that the model can use

# function inputs: data series, size of the window,The size of the batches to use when training,
# the size of the shuffle buffer, which determines how the data will be shuffled.
# Expand the dimensions of the series
# Create dataset ds from the series
# Slice the data up into the appropriate windows, shifted by one time set.
# keep them all the same size by setting drop remainder to true.
# flatten the data into numpy array in the size of our window_size + 1.
# shuffle data so that its not in order
# than split data into features and labels  

def windowed_dataset(series, window_size, batch_size, shuffle_buffer_size):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer_size)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [11]:
# helper function that can preform forecasting after training model
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(50).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [12]:
window_size

In [13]:
# used to find the optimum learning rate for the optimizer (form of hyperparameter tuning) using conv1D, LSTM, and dense layer
tf.keras.backend.clear_session() # this helps avoid clutter from old models and layers
tf.random.set_seed(51)  # to get reproducable results 
np.random.seed(51)  # generate pseudo-random numbers 

train_set = windowed_dataset(train_series, window_size, batch_size, shuffle_buffer_size)
print(train_set)
print(train_series.shape)

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=32, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 100) # scale outputs by 100 using lambda layer 
])

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20)) # to tune the learning rate set up a callback 

optimizer = tf.keras.optimizers.SGD(learning_rate=1e-8, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

In [14]:
# plot the learning rate loss 
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 1e-3, 0, 30])

In [15]:
# optimum learning rate found was headed towards 1e-4 
tf.keras.backend.clear_session() # this helps avoid clutter from old models and layers
tf.random.set_seed(51)
np.random.seed(51)
#batch_size = 16
dataset = windowed_dataset(train_series, window_size, batch_size, shuffle_buffer_size)

model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=32, kernel_size=3,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  tf.keras.layers.LSTM(32, return_sequences=True),
  tf.keras.layers.LSTM(32, return_sequences=True),
  tf.keras.layers.Dense(1),
  tf.keras.layers.Lambda(lambda x: x * 100)
])

optimizer = tf.keras.optimizers.SGD(learning_rate=1e-4, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(dataset,epochs=100)

In [16]:
window_size = 1
forecast = model_forecast(model, test_series[..., np.newaxis], window_size)
forecast = forecast[window_size:-1, -1, 0]
print(forecast.shape)

In [17]:
plt.figure(figsize=(10, 6))
plt.plot(test_time, test_series) #  plot test time and its corrsponding meantep as test_series
plt.plot(test_time[:112], forecast) #  plot test time first 112 and the predicted meantep 
plt.show()

In [18]:
print("First 7 Predictions :","\n", forecast[:7])
print('')
print('Actual first 7 values:', "\n", test_series[:7])

In [19]:
print('Mean absolute error:')
tf.keras.metrics.mean_absolute_error(test_series[:112], forecast).numpy()

In [20]:
# predictions on the left and actual values on the right 
m_p = pd.Series(forecast,test_series[:112])
m_p